<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/rating/rating_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training Tensorflow MobileNetSSD v2 and Inception v3 models to classify maps, phylogenies, illustrations, and herbarium sheets from EOL images
---
*Last Updated 23 December 2020*   
Train [MobileNet SSD v2](https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4) and [Inception v3](https://tfhub.dev/google/imagenet/inception_v3/classification/4) to rating EOL image quality from 1 (bad) to 5 (good). The training dataset is from EOL user-generated image ratings. Classifications will be used to generate image tags to improve sorting of EOLv3 images from high to low quality.

Training images were downloaded to Google Drive and processed using [rating_preprocessing.ipynb](https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/classification_for_image_tagging/rating/rating_preprocessing.ipynb). 

Pre-trained MobileNet SSD v2 and Inception v3 models were fine-tuned (via unfreezing lower layers) to classify image ratings. 

**Best results for each model from 60+ trials (some using early-stopping because of long train times and large detasets):**   
Upon inspection of training data, there was too much inconsistency for humans to reliably predict image ratings that matched EOL users. For future rating classifiers, it is expected that creating a curated dataset would give better results.
* MobileNet SSD v2 was trained for 12 hours to 10 epochs with Batch Size=16, Lr=0.001, Dropout=0.2. Final validation accuracy = 0.55
* **Inception v3 was trained for 12 hours to 10 epochs with Batch Size=32 Lr=0.001, Dropout=0. Final validation accuracy = 0.60**

**Notes**
* Change filepaths or information using the form fields to the right of code blocks (also noted in code with 'TO DO')
* Make sure to set the runtime to GPU Hardware Accelerator with a High Ram Runtime Shape (Runtime -> Change runtime type)

**References**   
* https://www.tensorflow.org/tutorials/customization/custom_training_walkthrough
* https://www.tensorflow.org/tutorials/images/classification
* https://medium.com/analytics-vidhya/create-tensorflow-image-classification-model-with-your-own-dataset-in-google-colab-63e9d7853a3e
* https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_image_retraining.ipynb#scrollTo=umB5tswsfTEQ
* https://medium.com/analytics-vidhya/how-to-do-image-classification-on-custom-dataset-using-tensorflow-52309666498e
* https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
* https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

## Imports   
---

In [ ]:
# Mount google drive to import/export files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# For working with data and plotting graphs
import itertools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# For image classification and training
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, InputLayer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

## Train Classification Model(s)
---

### Training Dataset Preparation

Use dropdown menu on the right to choose which pre-trained model to use and set the image batch size for training

In [ ]:
# TO DO: Select pre-trained model to use from Tensorflow Hub Model Zoo
module_selection = ("inception_v3", 299) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection

if handle_base == "inception_v3":
  MODULE_HANDLE ="https://tfhub.dev/google/imagenet/inception_v3/classification/4".format(handle_base)
  epsilon = 1
elif handle_base == "mobilenet_v2_1.0_224":
  MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4".format(handle_base) 
  epsilon = 1e-07
# TO DO: adjust batch size to make training faster or slower
BATCH_SIZE = "32" #@param ["16", "32", "64", "128"]

IMAGE_SIZE = (pixels, pixels)
print("Using {} with input size {} and batch size {}".format(handle_base, IMAGE_SIZE, BATCH_SIZE))

Make sure the second line does not say "Found 0 images belonging to..." and run again until a non-zero number is given to build a test dataset. Some images aren't read properly by ImageDataGenerator and they can be skipped during training without problems, but will result in no training class being generated during this step.

In [ ]:
# To suppress warnings from pillow about image sizes
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 95000000

# TO DO: Change directory to wherever images are stored
PATH = '/content/drive/My Drive/summer20/classification/rating/images/agg' #@param {type:"string"}
TRAINING_DATA_DIR = str(PATH)
print(TRAINING_DATA_DIR)

# TO DO: Adjust interpolation method and see how training results change
interpolation = "nearest" #@param ["nearest", "bilinear"]

# Set data generation and flow parameters
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=int(BATCH_SIZE),
                    interpolation = interpolation, color_mode='rgb')

# Make test dataset
test_datagen = ImageDataGenerator(**datagen_kwargs)
test_generator = test_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="validation",
shuffle=True,
**dataflow_kwargs
)

# Make train dataset
train_datagen = ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR, subset="training", shuffle=True, **dataflow_kwargs)

# Learn more about data batches
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

### Model Preparation

In [ ]:
# Build model
print("Building model with", handle_base)
# TO DO: If model is overfitting, add/increase dropout rate
dropout_rate = 0.1 #@param {type:"slider", min:0, max:0.5, step:0.1}
lr = 0.001 #@param ["0.1", "0.01", "0.001", "0.0001", "0.00001"] {type:"raw"}
# Freeze or unfreeze lower layers for transfer learning and fine tuning
trainable = True #@param ["True", "False"] {type:"raw"} 

def create_model():
  model = tf.keras.Sequential([
    InputLayer(input_shape=IMAGE_SIZE + (3,)),
    hub.KerasLayer(MODULE_HANDLE, trainable=trainable), #False freezes lower layers so only top classifier is retrained
    Dropout(rate = dropout_rate), 
    Dense(train_generator.num_classes, kernel_regularizer=tf.keras.regularizers.l2(0.0001))
  ])
  
  # Build model
  model.build((None,)+IMAGE_SIZE+(3,))
  
  # Compile model
  model.compile(
    # Parameters for Adam optimizer
    optimizer=tf.keras.optimizers.Adam(
      learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon, amsgrad=False,
      name='Adam'), 
      # Categorical cross entropy because 5 exclusive classes
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0),
    metrics=['accuracy'])
  return model

# Create new model instance
model = create_model()

# Steps per epoch and testing
steps_per_epoch = train_generator.samples // train_generator.batch_size
test_steps = test_generator.samples // test_generator.batch_size

# Display model architecture
model.summary()
tf.keras.utils.plot_model(model, show_shapes=True)

### Actual Training 

In [ ]:
# TO DO: Resume training from saved checkpoint?
resume = "N" #@param ["Y", "N"]
TRAIN_SESS_NUM = "12" #@param {type:"string"}

# TO DO: Adjust number of epochs to find balance between underfit and overfit for training
num_epochs = '10' #@param {type:"string"}

if resume == 'Y':
  TRAIN_SESS_NUM = TRAIN_SESS_NUM
  CKPTS_PATH = '/content/drive/My Drive/summer20/classification/rating/saved_models/' + TRAIN_SESS_NUM + '/ckpt/'
  CKPT_PATH = CKPTS_PATH + 'cp-{epoch:04d}.ckpt'
  latest = tf.train.latest_checkpoint(CKPTS_PATH)
  print("Restoring weights from Model {}, Checkpoint {}".format(TRAIN_SESS_NUM, latest))
  model.load_weights(latest)

else:
  # Save each new training attempt results in new folder
  last_attempt = !ls /content/drive/'My Drive'/summer20/classification/rating/saved_models/ | tail -n 1
  if not last_attempt:
    last_attempt = 0
  else:
    last_attempt = int(last_attempt.n)
    if last_attempt < 9:
      TRAIN_SESS_NUM = "0" + str(last_attempt + 1)
    else:
      TRAIN_SESS_NUM = str(last_attempt + 1)
  CKPT_PATH = '/content/drive/My Drive/summer20/classification/rating/saved_models/' + TRAIN_SESS_NUM + '/ckpt/cp-{epoch:04d}.ckpt' 

  print("Last training attempt number:", last_attempt)
  print("Training attempt number: {}, for {} epochs".format(TRAIN_SESS_NUM, num_epochs))
  # Save weights for 0th epoch
  model.save_weights(CKPT_PATH.format(epoch=0))

# Create a callback that saves the model's weights during training
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CKPT_PATH,
                                                 save_weights_only=True,
                                                 verbose=1) #verbose: Integer. 0, 1, or 2. Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch

# Train the model with the new callback
hist = model.fit(
    train_generator,
    epochs=int(num_epochs), steps_per_epoch=steps_per_epoch,
    callbacks=[ckpt_callback],
    validation_data=test_generator,
    validation_steps=test_steps).history

# Save trained model 
saved_model_path = '/content/drive/My Drive/summer20/classification/rating/saved_models/' + TRAIN_SESS_NUM
tf.saved_model.save(model, saved_model_path)

#### Plot loss and accuracy for training session

In [ ]:
# Plot loss
plt.figure()
plt.title("Attempt {}:Training and Validation Loss".format(TRAIN_SESS_NUM))
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(hist["loss"], label='Train')
plt.plot(hist["val_loss"], label='Test')
plt.legend(loc='lower right')

# Plot accuracy
plt.figure()
plt.title("Attempt {}:Training and Validation Accuracy".format(TRAIN_SESS_NUM))
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(hist["accuracy"], label='Train')
plt.plot(hist["val_accuracy"], label='Test')
plt.legend(loc='upper right')
path = '/content/drive/My Drive/summer20/classification/rating/train_graphs/' + TRAIN_SESS_NUM + '.png'
plt.savefig(path)

# Print final loss and accuracy values
final_loss, final_accuracy = model.evaluate(test_generator, steps = test_steps)
print('Final loss: {:.2f}'.format(final_loss))
print('Final accuracy: {:.2f}%'.format(final_accuracy * 100))

## Review training results
---   
Display classification results on images

In [ ]:
# Define functions

# TO DO: Do you want to display classification results for the most recently trained model?
answer = "Yes" #@param ["Yes", "No"]
# TO DO: If No, manually input desired training attempt number to the right
if answer == "Yes":
  # Display results from most recent training attempt
  last_attempt = !ls /content/drive/'My Drive'/summer20/classification/rating/saved_models/ | tail -n 1
  TRAIN_SESS_NUM = str(last_attempt.n)
else:
  TRAIN_SESS_NUM = "20" #@param ["13", "11", "15"] {allow-input: true}

# Load trained model from path
saved_model_path = '/content/drive/My Drive/summer20/classification/rating/saved_models/' + TRAIN_SESS_NUM
imtype_model = tf.keras.models.load_model(saved_model_path)

# TO DO: Select model type
module_selection = ("inception_v3", 299) #@param ["(\"mobilenet_v2_1.0_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
IMAGE_SIZE = (pixels, pixels)

# Function for plotting classification results with color-coded label if true or false prediction
label_names = ['bad', 'good']

In [ ]:
# Run inference
from PIL import Image
import time

# TO DO: Choose which image class to inspect results for in true_imclass to right
# TO DO: Choose start and end image numbers to inspect (inspect up to 50 images at a time)
base = '/content/drive/My Drive/summer20/classification/'
classifier = "rating/" #@param ["flower_fruit/", "image_type/", "rating/"]
true_imclass = "agg/good" #@param ["agg/good", "agg/bad"]
PATH_TO_TEST_IMAGES_DIR = base + classifier + "images/" + true_imclass
names = os.listdir(PATH_TO_TEST_IMAGES_DIR)
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR, name) for name in names]

# Loops through first 5 image urls from the text file
start = 250 #@param {type:"number"}
end =   300#@param {type:"number"}
for im_num, im_path in enumerate(TEST_IMAGE_PATHS[start:end], start=1):
    # Load in image
    imga = Image.open(im_path)
    img = imga.convert('RGB')
    image = img.resize(IMAGE_SIZE)
    image = np.reshape(image,[1,pixels,pixels,3])
    image = image*1./255
    # Record inference time
    start_time = time.time()
    # Detection and draw boxes on image
    predictions = imtype_model.predict(image, batch_size=1)
    label_num = np.argmax(predictions[0], axis=-1)
    conf = predictions[0][label_num]
    imclass = label_names[label_num]
    end_time = time.time()
    # Display progress message after each image
    print('Inference complete for {} of {} images'.format(im_num, (end-start)))
    # Plot and show detection boxes on images
    _, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(img)
    plt.title('{}) Prediction: {}, Confidence: {}, Inference time: {}'.format(im_num, imclass, \
    format(conf, '.2f'), format(end_time-start_time, '.2f')))